In [ ]:
import cv2
import numpy as np
import tensorflow as tf

# Load your custom .h5 model using TensorFlow or Keras
custom_model = tf.keras.models.load_model("animal_recognition_model.h5")

# Define the class names (modify this based on your custom model's classes)
class_names = ['Armadilles', 'Bear', 'Birds', 'Cow', 'Crocodile', 'Deer', 'Elephant', 'Goat', 'Horse', 'Jaguar', 'Monkey', 'Rabbit', 'Skunk', 'Tiger', 'Wild Boar']

# Function to perform object detection on a single image frame using your custom model
def detect_objects(frame):
    # Preprocess the frame (resize, normalize, etc.) based on your model's requirements
    input_width = 224  # Replace with your model's input width
    input_height = 224  # Replace with your model's input height

    # Resize and normalize the frame
    resized_frame = cv2.resize(frame, (input_width, input_height))
    normalized_frame = resized_frame / 255.0  # Normalize pixel values to [0, 1]

    # Perform object detection using your custom model
    predictions = custom_model.predict(np.expand_dims(normalized_frame, axis=0))

    # Process the predictions to get bounding boxes, class probabilities, and class indices
    if len(predictions.shape) == 2:
        # Assuming the model output is 2D with the format [x_min, y_min, x_max, y_max, confidence, class_index]
        bounding_boxes = predictions[:, :4]
        class_probs = predictions[:, 4]
        class_indices = predictions[:, 5].astype(int)
    elif len(predictions.shape) == 3:
        # Assuming the model output is 3D with the format [[x_min, y_min, x_max, y_max, confidence, class_index]]
        bounding_boxes = predictions[0, :, :4]
        class_probs = predictions[0, :, 4]
        class_indices = predictions[0, :, 5].astype(int)
    else:
        raise ValueError("Unsupported model output format")

    # Assuming bounding_boxes is a list of [x_min, y_min, x_max, y_max] format bounding boxes
    # Convert them to integers and scale back to the original frame size
    frame_height, frame_width, _ = frame.shape
    bounding_boxes = [
        (
            int(x_min * frame_width),
            int(y_min * frame_height),
            int(x_max * frame_width),
            int(y_max * frame_height),
        )
        for x_min, y_min, x_max, y_max in bounding_boxes
    ]

    # Assuming class_probs is a list of probabilities for each class
    # Assuming class_indices is a list of integer class indices (0, 1, 2, ...)
    # Find the class with the highest probability and its corresponding index
    max_class_prob = max(class_probs)
    max_class_index = class_indices[np.argmax(class_probs)]

    # Get the class name based on the max_class_index
    predicted_class = class_names[max_class_index]

    # Draw bounding boxes and labels on the frame
    for (x_min, y_min, x_max, y_max) in bounding_boxes:
        color = (0, 255, 0)  # Green color for bounding box
        cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), color, 2)
        label = f"{predicted_class} {max_class_prob:.2f}"
        cv2.putText(frame, label, (x_min, y_min - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    return frame

video_file = "cow.mp4"

# Output video file
output_file = "output_video.avi"

# Open video file
video_capture = cv2.VideoCapture(video_file)

# Get video properties
frame_width = int(video_capture.get(3))
frame_height = int(video_capture.get(4))
fps = int(video_capture.get(cv2.CAP_PROP_FPS))

# Define codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(output_file, fourcc, fps, (frame_width, frame_height))

# Process each frame in the video
while True:
    ret, frame = video_capture.read()
    if not ret:
        break

    # Perform object detection on the current frame
    frame = detect_objects(frame)

    # Write the processed frame to the output video
    out.write(frame)

    # Display the frame (optional, for visualization)
    cv2.imshow("Object Detection", frame)

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# Release video capture and writer objects
video_capture.release()
out.release()

# Destroy any OpenCV windows
cv2.destroyAllWindows()


1/1 [==============================] - 0s 40ms/step


1/1 [==============================] - 0s 44ms/step
